# Use GRPO to Fine-Tune Qwen2.5-0.5B-Instruct with TRL

This notebook explains how you can use GRPO with the help of Hugging Face [TRL](https://huggingface.co/docs/trl/index) on Amazon SageMaker. 

**This notebook is validated and optimized to run on `ml.p4d.24xlarge` instances**


Hugging Face shares the support of GRPO and PyTorch FSDP (Fully Sharded Data Parallel). FSDP and GRPO allow you now to fine-tune foundation models.

* [PyTorch FSDP](https://pytorch.org/blog/introducing-pytorch-fully-sharded-data-parallel-api/) is a data/model parallelism technique that shards model across GPUs, reducing memory requirements and enabling the training of larger models more efficiently​​​​​​.
* GRPO is a training method that leverages Rienforcment learning to fine-tune and/or train foundation model

This notebook walks you thorugh how to fine-tune open LLMs from Hugging Face using Amazon SageMaker.

## 1. Setup Development Environment

Our first step is to install Hugging Face Libraries we need on the client to correctly prepare our dataset and start our training/evaluations jobs. 

In [2]:
! pip install transformers "datasets[s3]==2.18.0" "sagemaker>=2.190.0" --upgrade --quiet

Log in huggingface hub to get data

In [3]:
from huggingface_hub import login

login(token="") # ADD YOUR HF TOKEN HERE

If you are going to use Sagemaker in a local environment. You need access to an IAM Role with the required permissions for Sagemaker. You can find [here](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-roles.html) more about it.

In [4]:
import sagemaker
from datasets import load_dataset
import pandas as pd
from sagemaker.pytorch import PyTorch
from transformers import AutoTokenizer
import boto3
import os

sagemaker_session = sagemaker.Session()
bucket_name = sagemaker_session.default_bucket()

/opt/conda/lib/python3.12/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


[04/28/25 16:36:30] INFO     PyTorch version 2.5.1 available.                                          ]8;id=149223;file:///opt/conda/lib/python3.12/site-packages/datasets/config.py\config.py]8;;\:]8;id=925262;file:///opt/conda/lib/python3.12/site-packages/datasets/config.py#58\58]8;;\

                    INFO     TensorFlow version 2.18.0 available.                                      ]8;id=37056;file:///opt/conda/lib/python3.12/site-packages/datasets/config.py\config.py]8;;\:]8;id=417656;file:///opt/conda/lib/python3.12/site-packages/datasets/config.py#95\95]8;;\

In [5]:
sess = sagemaker.Session()
sagemaker_session_bucket=None

if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sagemaker_session = sagemaker.Session(default_bucket=sagemaker_session_bucket)

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sagemaker_session.default_bucket()}")
print(f"sagemaker session region: {sagemaker_session.boto_region_name}")

sagemaker role arn: arn:aws:iam::783764584149:role/service-role/AmazonSageMaker-ExecutionRole-20241230T144802
sagemaker bucket: sagemaker-us-east-1-783764584149
sagemaker session region: us-east-1


# 2. Create and prepare the dataset

In this example, we use the mathematic data to teach the model mathematical reasoning. We are going to use the [Jiayi-Pan/Countdown-Tasks-3to4](https://huggingface.co/datasets/Jiayi-Pan/Countdown-Tasks-3to4) dataset, which contains samples with 3 to 4 numbers and solutions.


In [6]:
from transformers import AutoTokenizer
from datasets import load_dataset

# Load dataset from Hugging Face Hub
dataset_id = "Jiayi-Pan/Countdown-Tasks-3to4"
dataset = load_dataset(dataset_id, split="train")
# select a random subset of 50k samples
dataset = dataset.shuffle(seed=42)


In [7]:
# split the dataset into train and test
train_test_split = dataset.train_test_split(test_size=0.1)

train_dataset = train_test_split["train"]
test_dataset = train_test_split["test"]

We are going to use the [FileSystem integration](https://huggingface.co/docs/datasets/filesystems) to upload our dataset to S3. We are using the `sagemaker_session.default_bucket()`, adjust this if you want to store the dataset in a different S3 bucket. We will use the S3 path later in our training script.

In [8]:
# save train_dataset to s3 using our SageMaker session
input_path = f's3://{sagemaker_session.default_bucket()}/datasets/grpo-sm-test'

# Save datasets to s3
# We will fine tune only with 20 records due to limited compute resource for the workshop
train_dataset.to_json(f"{input_path}/train/dataset.json", orient="records")
train_dataset_s3_path = f"{input_path}/train/dataset.json"
test_dataset.to_json(f"{input_path}/test/dataset.json", orient="records")
test_dataset_s3_path = f"{input_path}/test/dataset.json"
print(f"Training data uploaded to:")
print(train_dataset_s3_path)
print(f"Test data uploaded to:")
print(test_dataset_s3_path)
print(f"https://s3.console.aws.amazon.com/s3/buckets/{sagemaker_session.default_bucket()}/?region={sagemaker_session.boto_region_name}&prefix={input_path.split('/', 3)[-1]}/")


Creating json from Arrow format:   0%|          | 0/442 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/50 [00:00<?, ?ba/s]

Training data uploaded to:
s3://sagemaker-us-east-1-783764584149/datasets/grpo-sm-test/train/dataset.json
Test data uploaded to:
s3://sagemaker-us-east-1-783764584149/datasets/grpo-sm-test/test/dataset.json
https://s3.console.aws.amazon.com/s3/buckets/sagemaker-us-east-1-783764584149/?region=us-east-1&prefix=datasets/grpo-sm-test/


## 3. GRPO on Amazon SageMaker using python sdk

We are now ready to fine-tune our model. We will use the [GRPOTrainer](https://huggingface.co/docs/trl/main/en/grpo_trainer) from `trl` to train our model. The `GRPOTrainer` is a subclass of the `Trainer` from the `transformers`. We prepared a script [train.py](../scripts/train.py) which will loads the dataset from disk, prepare the model, tokenizer and start the training. It usees the [GRPOTrainer](https://huggingface.co/docs/trl/main/en/grpo_trainer) from `trl` to ftrain our model. 

For configuration we use `TrlParser`, that allows us to provide hyperparameters in a yaml file. This `yaml` will be uploaded and provided to Amazon SageMaker similar to our datasets. Below is the config file for GRPO on ml.p4d.24xlarge 40GB GPUs. We are saving the config file as `args.yaml` and upload it to S3.


In [9]:
%%bash

cat > ./args.yaml <<EOF
hf_token: "" # Use HF token to login into Hugging Face to access the DeepSeek distilled models
wandb_token: ""
model_id: "Qwen/Qwen2.5-0.5B-Instruct"      # Hugging Face model id, replace it with 70b if needeed
max_seq_length: 1024  #512 # 2048               # max sequence length for model and packing of the dataset
# sagemaker specific parameters
train_dataset_path: "/opt/ml/input/data/train/" # path to where SageMaker saves train dataset
test_dataset_path: "/opt/ml/input/data/test/"   # path to where SageMaker saves test dataset

output_dir: "/opt/ml/model/Qwen-GRPO/output"              # path to where SageMaker will upload the model 
# training parameters
report_to: "wandb"             # report metrics to wandb
learning_rate: 0.0003                  # learning rate 2e-4
lr_scheduler_type: "constant"          # learning rate scheduler
num_train_epochs: 1                  # number of training epochs
per_device_train_batch_size: 10       # batch size per device during training
per_device_eval_batch_size: 8         # batch size for evaluation
gradient_accumulation_steps: 2        # number of steps before performing a backward/update pass
optim: adamw_torch                     # use torch adamw optimizer
logging_steps: 10                      # log every 10 steps
save_strategy: epoch                   # save checkpoint every epoch
evaluation_strategy: epoch             # evaluate every epoch
max_grad_norm: 0.3                     # max gradient norm
warmup_ratio: 0.03                     # warmup ratio
bf16: true                             # use bfloat16 precision
tf32: true                             # use tf32 precision
gradient_checkpointing: true           # use gradient checkpointing to save memory

weight_decay: 0.01
warmup_steps: 100
# offload FSDP parameters: https://huggingface.co/docs/transformers/main/en/fsdp
fsdp: "full_shard auto_wrap" # remove offload if enough GPU memory
fsdp_config:
  backward_prefetch: "backward_pre"
  forward_prefetch: "false"
  use_orig_params: "false"
EOF

In [10]:
from sagemaker.s3 import S3Uploader

# upload the model yaml file to s3
model_yaml = "args.yaml"
train_config_s3_path = S3Uploader.upload(local_path=model_yaml, desired_s3_uri=f"{input_path}/config")

print(f"Training config uploaded to:")
print(train_config_s3_path)

Training config uploaded to:
s3://sagemaker-us-east-1-783764584149/datasets/grpo-sm-test/config/args.yaml


In [11]:
# image_uri = (
#     f"658645717510.dkr.ecr.{sagemaker_session.boto_session.region_name}.amazonaws.com/smdistributed-modelparallel:2.4.1-gpu-py311-cu121"
# )
image_uri = (
    f'763104351884.dkr.ecr.{sagemaker_session.boto_session.region_name}.amazonaws.com/pytorch-training:2.6.0-gpu-py312-cu126-ubuntu22.04-sagemaker'
)

image_uri

image_uri

'763104351884.dkr.ecr.us-east-1.amazonaws.com/pytorch-training:2.6.0-gpu-py312-cu126-ubuntu22.04-sagemaker'

In [12]:
# Create SageMaker PyTorch Estimator

# define Training Job Name 
job_name = f'Qwen-GRPO'

pytorch_estimator = PyTorch(
    entry_point= 'train.py',
    source_dir="./scripts",
    job_name=job_name,
    base_job_name=job_name,
    max_run=10800,
    role=role,
    #framework_version="2.2.0",
    image_uri = image_uri,
    py_version="py310",
    instance_count=1,
    instance_type="ml.p4d.24xlarge",
    sagemaker_session=sagemaker_session,
    disable_output_compression=True,
    keep_alive_period_in_seconds=1800,
    distribution={"torch_distributed": {"enabled": True}},
    hyperparameters={
        "config": "/opt/ml/input/data/config/args.yaml" # path to TRL config which was uploaded to s3
    }
)

In [13]:
# define a data input dictonary with our uploaded s3 uris
data = {
  'train': train_dataset_s3_path,
  'test': test_dataset_s3_path,
  'config': train_config_s3_path
  }

# Check input channels configured 
data

{'train': 's3://sagemaker-us-east-1-783764584149/datasets/grpo-sm-test/train/dataset.json',
 'test': 's3://sagemaker-us-east-1-783764584149/datasets/grpo-sm-test/test/dataset.json',
 'config': 's3://sagemaker-us-east-1-783764584149/datasets/grpo-sm-test/config/args.yaml'}

In [ ]:
# starting the train job with our uploaded datasets as input
pytorch_estimator.fit(data, wait=True)

[04/28/25 17:36:39] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=978841;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=604538;file:///opt/conda/lib/python3.12/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: Qwen-GRPO-2025-04-28-17-36-39-247     ]8;id=226017;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=99729;file:///opt/conda/lib/python3.12/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-04-28 17:36:39 Starting - Starting the training job.